In [1]:
#PART1 - Converting WIKI table into PANDA DATAFRAME
# Import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup 
# Webscrapping the URL
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
print(page.status_code)
soup = BeautifulSoup(page.text,"html.parser")

200


In [2]:
# READ Table
Table_array = []
Table_text_element = soup.find_all( class_= "wikitable sortable")
Table_text_element=Table_text_element[0]
for row in Table_text_element.find_all('tr'):
        for cell in row.find_all('td'):
             Table_array.append(cell.text)
Table_array1string=', '.join(Table_array)
Table_array1string = 'Column,PostCode,Borough,Neighbourhood' + '\n, '+ Table_array1string
fh = open ('Testfile.txt', 'w',)
fh.write(Table_array1string)
fh.close()
df = pd.read_csv('Testfile.txt')
#Remove the unwanted Column as part of Cleansing
df.drop('Column',axis=1,inplace=True)

In [4]:
# q1 Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
# Creating new dataframe excluding column - Borough having value 
df_q1= df[df['Borough'] != ' Not assigned']
print(df_q1.shape)
df_q1.head()

(211, 3)


,PostCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [5]:
#Q3 If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
#Replacing Neighbourhood value with borough if its ' Not assigned'
df_q1.loc[df_q1['Neighbourhood'] == ' Not assigned', 'Neighbourhood'] = df_q1.Borough
print (df_q1.shape)

#Q2 More than one neighborhood can exist in one postal code area. 
#For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: 
#Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as 
#shown in row 11 in the above table.

res= df_q1.groupby(['PostCode','Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)]).reset_index()
print("Number of rows in the final dataframe as per .Shape ", res.shape)

res.columns = ['Postal Code', 'Borough', 'Neighbourhood']
res.head() 
res.tail()
# END OF PART1

(211, 3)
Number of rows in the final dataframe as per .Shape  (103, 3)


/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Postal Code,Borough,Neighbourhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Ri..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergat..."
102,M9W,Etobicoke,Northwest


In [8]:
# PART 2 - MERGING IT WITH GEOSPATIAL DATA BEING PROVIDED
# Reading it from the geo file -http://cocl.us/Geospatial_data
print(type(res))
url = "http://cocl.us/Geospatial_data"
geodf = pd.read_csv(url)
geodf.columns = ['GeoPostCode','Latitude', 'Longitude']


<class 'pandas.core.frame.DataFrame'>


In [9]:
# Quick verification & Comparison with the earlier data frame  res in Part1 , before we merge it.
print(geodf.head())
print(geodf.tail())


  GeoPostCode   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
    GeoPostCode   Latitude  Longitude
98          M9N  43.706876 -79.518188
99          M9P  43.696319 -79.532242
100         M9R  43.688905 -79.554724
101         M9V  43.739416 -79.588437
102         M9W  43.706748 -79.594054


In [55]:
Concatdf = pd.concat([res,geodf], axis=1)
finalgeodf=Concatdf.drop(['GeoPostCode'] , axis=1)
finalgeodf.head()
# End of Part2

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [ ]:
# PART 3 - 